In [1]:
# import scipy
# from scipy.optimize import curve_fit
# from scipy.io import wavfile
# from scipy.signal import butter, sosfiltfilt
# import matplotlib.pyplot as plt
# import pandas as pd
# import numpy as np
# from scipy import signal
# import osascript
# from gtts import gTTS 
# from matplotlib import pylab
# import os 
# import pyaudio
# import wave
# import keyboard  # using module keyboard
# import soundfile as sf
# import math
# import pyloudnorm as pyln
# from sys import byteorder
# from array import array
# from struct import pack
# import import_ipynb
# import concat_project7
# from joblib import dump, load
from emotion_recognition import EmotionRecognizer
from sklearn.svm import SVC
from deep_emotion_recognition import DeepEmotionRecognizer

W0815 14:50:51.188915 140736091374464 deprecation_wrapper.py:119] From /Users/m.elabd/Desktop/loudness/Emote-X/emotion-recognition-using-speech/deep_emotion_recognition.py:15: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.



# Constants

In [ ]:
# THRESHOLD = 100
# CHUNK_SIZE = 2048
# FORMAT = pyaudio.paInt16
# RATE = 16000
# AVG_STEP = 75
# PERCENTILE = 70
# BANDPASS_FREQ = [300, 3400]
# CALIBRATION_VOLUME = 40

# Record Audio

#### Record from microphone

In [ ]:
def record():
    p = pyaudio.PyAudio()
    stream = p.open(format=FORMAT, channels=1, rate=RATE,
        input=True, output=True,
        frames_per_buffer=CHUNK_SIZE)

    num_silent = 0
    snd_started = False

    r = array('h')

    while 1:
        # little endian, signed short
        snd_data = array('h', stream.read(CHUNK_SIZE))
        if byteorder == 'big':
            snd_data.byteswap()
        r.extend(snd_data)

        silent = is_silent(snd_data)

        if silent and snd_started:
            num_silent += 1
        elif not silent and not snd_started:
            snd_started = True

        if snd_started and num_silent > 60:
            break

    sample_width = p.get_sample_size(FORMAT)
    stream.stop_stream()
    stream.close()
    p.terminate()

#     r = trim(r)
#     r = add_silence(r, 0.5)
    return sample_width, r

#### Save to wav file

In [ ]:
def record_to_file(path):
    sample_width, data = record()
    data = pack('<' + ('h'*len(data)), *data)

    wf = wave.open(path, 'wb')
    wf.setnchannels(1)
    wf.setsampwidth(sample_width)
    wf.setframerate(RATE)
    wf.writeframes(data)
    wf.close()

#### Checks if input is silent based on threshold

In [ ]:
def is_silent(snd_data):
    "Returns 'True' if below the 'silent' threshold"
    return max(snd_data) < THRESHOLD

# Preprocessing

#### Trim input to remove silence at beginning and end

In [ ]:
def trim(snd_data):
    def _trim(snd_data):
        snd_started = False
        r = array('h')

        for i in snd_data:
            if not snd_started and abs(i)>THRESHOLD:
                snd_started = True
                r.append(i)
            elif snd_started:
                r.append(i)
        return r

    # Trim to the left
    snd_data = _trim(snd_data)

    # Trim to the right
    snd_data.reverse()
    snd_data = _trim(snd_data)
    snd_data.reverse()
    return snd_data

#### Pad with silence at beginning and end

In [ ]:
def add_silence(snd_data, seconds):
    ""
    r = array('h', [0 for i in range(int(seconds*RATE))])
    r.extend(snd_data)
    r.extend([0 for i in range(int(seconds*RATE))])
    return r

#### Process wav input with highpass filter

In [ ]:
def voice_input(voice_data):
    fs, data = wavfile.read(voice_data)
    data = butter_bandpass_filter(data, BANDPASS_FREQ[0], BANDPASS_FREQ[1], fs)
    return data, fs

#### Implement butter bandpass filter

In [ ]:
# def butter_bandpass(lowcut, highcut, fs, order=5):
#     nyq = 0.5 * fs
#     low = lowcut / nyq
#     high = highcut / nyq
#     sos = butter(order, [low, high], btype='band', analog=False, output='sos')
#     return sos

# def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
#     sos = butter_bandpass(lowcut, highcut, fs, order=order)
#     y = sosfiltfilt(sos, data)
#     return y

# Feature Extraction

# Emotion Algorithm

#### Get amplitude curve and apply regression on ITU-R BS.1770-4 loudness algorithm

In [ ]:
# def getting_emotion(data, rate):

    
# #     lol_data, fs = concat_project7.clean_sound(data, rate)

#     #Convert raw data into features
#     print("Data acquired")

#     feature_data = concat_project7.get_all_features(data, fs);
    
#     #return the exponential regression
#     return (feature_data)

#### Remove silence before, after, and in-between voice data

In [ ]:
def remove_silence_from(amplitudes, threshold):
    silenced = []
    for x in amplitudes:
        if x >= threshold:
            silenced.append(x)
    return silenced# Preprocessing

#### Regression converts loudness to scale 0 to 100 and adjusts for calibration offset

# Calibration

In [ ]:
# def get_offset(calib_loudness):
#     return calib_loudness - CALIBRATION_VOLUME

# Response

#### Sets a new volume that is spoken at for the user

In [ ]:
def reply_volume(user_volume):
    speaker_volume = int(osascript.osascript('output volume of (get volume settings)')[1])
    print("speaker volume is " + str(speaker_volume))
    print("user volume is "+ str(user_volume))
    if(speaker_volume < user_volume):
        new_volume = speaker_volume + (((speaker_volume - user_volume)**2) /100)
    else:
        new_volume = speaker_volume - (((speaker_volume - user_volume)**2) /100)
    print("speaker new volume is " + str(new_volume))
    osascript.osascript("set volume output volume "+ str(new_volume)) 
    thing_to_say = 'hello your volume is at '+ str(int(new_volume))
    print(thing_to_say)
    os.system("say "+ thing_to_say)

In [ ]:
def reply_emotion(user_emotion):
    os.system("say "+" are you feeling "+user_emotion)

# Main

In [ ]:
deeprec = DeepEmotionRecognizer(emotions=['sad', 'neutral', 'happy', 'angry', 'disgust', 'fear'],
                                n_rnn_layers=2, n_dense_layers=3, rnn_units=256, dense_units=128,
                                dropout=0.35,epochs=500)
# train the model
deeprec.train(override=False)
# get the accuracy
print(deeprec.test_score())
print(deeprec.confusion_matrix())

In [ ]:
# os.system("say "+ "Please speak into the microphone")
# print("Please speak into the microphone")
# record_to_file('emotion.wav')
# print("done")
# prediction = deeprec.predict('emotion.wav')
# print(f"Prediction: {prediction}")
# reply_emotion(prediction)


In [ ]:
from emotion_recognition import EmotionRecognizer
from sklearn.svm import SVC
# init a model, let's use SVC
my_model = SVC()
# pass my model to EmotionRecognizer instance
# and balance the dataset
rec = EmotionRecognizer(model=my_model, emotions=['angry', 'happy', 'sad'], balance=True, verbose=0)
# train the model
rec.train()
# check the test accuracy for that model
print("Test score:", rec.test_score())
# check the train accuracy for that model
print("Train score:", rec.train_score())

In [ ]:
# loads the best estimators from `grid` folder that was searched by GridSearchCV in `grid_search.py`,
# and set the model to the best in terms of test score, and then train it
rec.determine_best_model(train=True)
# get the determined sklearn model name
print(rec.model.__class__.__name__, "is the best")
# get the test accuracy score for the best estimator
print("Test score:", rec.test_score())

In [ ]:
# os.system("say "+ "Please speak into the microphone")
# print("Please speak into the microphone")
# record_to_file('emotion.wav')
# print("done")
# prediction = rec.predict('emotion.wav')
# print(f"Prediction: {prediction}")
# reply_emotion(prediction)


In [ ]:
from emotion_recognition import EmotionRecognizer

import pyaudio
import os
import wave
from sys import byteorder
from array import array
from struct import pack
from sklearn.ensemble import GradientBoostingClassifier, BaggingClassifier

from utils import get_best_estimators

THRESHOLD = 500
CHUNK_SIZE = 1024
FORMAT = pyaudio.paInt16
RATE = 16000

SILENCE = 30

def is_silent(snd_data):
    "Returns 'True' if below the 'silent' threshold"
    return max(snd_data) < THRESHOLD

def normalize(snd_data):
    "Average the volume out"
    MAXIMUM = 16384
    times = float(MAXIMUM)/max(abs(i) for i in snd_data)

    r = array('h')
    for i in snd_data:
        r.append(int(i*times))
    return r

def trim(snd_data):
    "Trim the blank spots at the start and end"
    def _trim(snd_data):
        snd_started = False
        r = array('h')

        for i in snd_data:
            if not snd_started and abs(i)>THRESHOLD:
                snd_started = True
                r.append(i)

            elif snd_started:
                r.append(i)
        return r

    # Trim to the left
    snd_data = _trim(snd_data)

    # Trim to the right
    snd_data.reverse()
    snd_data = _trim(snd_data)
    snd_data.reverse()
    return snd_data

def add_silence(snd_data, seconds):
    "Add silence to the start and end of 'snd_data' of length 'seconds' (float)"
    r = array('h', [0 for i in range(int(seconds*RATE))])
    r.extend(snd_data)
    r.extend([0 for i in range(int(seconds*RATE))])
    return r

def record():
    """
    Record a word or words from the microphone and 
    return the data as an array of signed shorts.

    Normalizes the audio, trims silence from the 
    start and end, and pads with 0.5 seconds of 
    blank sound to make sure VLC et al can play 
    it without getting chopped off.
    """
    p = pyaudio.PyAudio()
    stream = p.open(format=FORMAT, channels=1, rate=RATE,
        input=True, output=True,
        frames_per_buffer=CHUNK_SIZE)

    num_silent = 0
    snd_started = False

    r = array('h')

    while 1:
        # little endian, signed short
        snd_data = array('h', stream.read(CHUNK_SIZE))
        if byteorder == 'big':
            snd_data.byteswap()
        r.extend(snd_data)

        silent = is_silent(snd_data)

        if silent and snd_started:
            num_silent += 1
        elif not silent and not snd_started:
            snd_started = True

        if snd_started and num_silent > SILENCE:
            break

    sample_width = p.get_sample_size(FORMAT)
    stream.stop_stream()
    stream.close()
    p.terminate()

    r = normalize(r)
    r = trim(r)
    r = add_silence(r, 0.5)
    return sample_width, r

def record_to_file(path):
    "Records from the microphone and outputs the resulting data to 'path'"
    sample_width, data = record()
    data = pack('<' + ('h'*len(data)), *data)

    wf = wave.open(path, 'wb')
    wf.setnchannels(1)
    wf.setsampwidth(sample_width)
    wf.setframerate(RATE)
    wf.writeframes(data)
    wf.close()


def get_estimators_name(estimators):
    result = [ '"{}"'.format(estimator.__class__.__name__) for estimator, _, _ in estimators ]
    return ','.join(result), {estimator_name.strip('"'): estimator for estimator_name, (estimator, _, _) in zip(result, estimators)}





# DEEP REC NN

In [2]:
if __name__ == "__main__":
    deeprec = DeepEmotionRecognizer(emotions=['sad', 'happy'],
                                    n_rnn_layers=2, n_dense_layers=3, rnn_units=256, dense_units=128,
                                    dropout=0.35,epochs=500)
    # train the model
    deeprec.train(override=False)
    # get the accuracy
    print(deeprec.test_score())

    print("Test accuracy score: {:.3f}%".format(deeprec.test_score()*100))
#     print("Please talk")
    
    
#     filename = "test.wav"
#     record_to_file(filename)
#     result = deeprec.predict(filename)
#     print(result)
# NOTES
# look for happy vs anger
# api to remove other people voice


Extracting features for train:   0%|          | 0/3837 [00:00<?, ?it/s]

[TESS&RAVDESS] There are 812 training audio files for category:sad
[TESS&RAVDESS] There are 146 testing audio files for category:sad
[TESS&RAVDESS] There are 805 training audio files for category:happy
[TESS&RAVDESS] There are 147 testing audio files for category:happy
[+] Writed TESS & RAVDESS DB CSV File
[EMO-DB] Total files to write: 133
[EMO-DB] Training samples: 106
[EMO-DB] Testing samples: 26
[+] Writed EMO-DB CSV File
[+] Writed Custom DB CSV File


UnboundLocalError: local variable 'chroma_cens' referenced before assignment

In [ ]:
## os.system("say "+ "Please speak into the microphone")
print("Please speak into the microphone")
record_to_file('emotion.wav')
print("done")
prediction, confidence = deeprec.predict('emotion.wav')
print(f"Prediction: {prediction}")
reply_emotion(prediction)
print("happy confidence: " + str(confidence[0][0][1]*100) + "%")
print("sad confidence: " + str(confidence[0][0][0]*100) + "%")


In [ ]:
if __name__ == "__main__":
    deeprec_angry = DeepEmotionRecognizer(emotions=['sad', 'happy', 'angry'],
                                    n_rnn_layers=2, n_dense_layers=3, rnn_units=256, dense_units=128,
                                    dropout=0.35,epochs=500)
    # train the model
    deeprec_angry.train(override=False)
    # get the accuracy
    print(deeprec_angry.test_score())

    print("Test accuracy score: {:.3f}%".format(deeprec_angry.test_score()*100))
#     print("Please talk")
    
    
#     filename = "test.wav"
#     record_to_file(filename)
#     result = deeprec.predict(filename)
#     print(result)
# NOTES
# look for happy vs anger
# api to remove other people voice


In [ ]:
## os.system("say "+ "Please speak into the microphone")
print("Please speak into the microphone")
record_to_file('emotion.wav')
print("done")
prediction, confidence = deeprec_angry.predict('emotion.wav')
print(f"Prediction: {prediction}")
reply_emotion(prediction)
print("happy confidence: " + str(confidence[0][0][1]*100) + "%")
print("sad confidence: " + str(confidence[0][0][0]*100) + "%")


In [ ]:
if __name__ == "__main__":
    deeprec = DeepEmotionRecognizer(emotions=['happy', 'sad'],
                        features=['mfcc', 'chroma','mel','contrast','tonnetz', 'rmse', 'poly_feature','chroma_cens','deriv'],
                                    balance=True, verbose=1,
                                    n_rnn_layers=2, n_dense_layers=3, rnn_units=128, dense_units=256,
                                    dropout=0.3,epochs=300)
    # train the model
    deeprec.train(override=False)
    # get the accuracy
    print(deeprec.test_score())

    print("Test accuracy score: {:.3f}%".format(deeprec.test_score()*100))
#     print("Please talk")
    
    
#     filename = "test.wav"
#     record_to_file(filename)
#     result = deeprec.predict(filename)
#     print(result)

Extracting features for train:   0%|          | 0/3837 [00:00<?, ?it/s]

[TESS&RAVDESS] There are 805 training audio files for category:happy
[TESS&RAVDESS] There are 147 testing audio files for category:happy
[TESS&RAVDESS] There are 812 training audio files for category:sad
[TESS&RAVDESS] There are 146 testing audio files for category:sad
[+] Writed TESS & RAVDESS DB CSV File
[EMO-DB] Total files to write: 133
[EMO-DB] Training samples: 106
[EMO-DB] Testing samples: 26
[+] Writed EMO-DB CSV File
[+] Writed Custom DB CSV File


Extracting features for train:   2%|▏         | 65/3837 [00:44<40:23,  1.56it/s] 

In [ ]:
os.system("say "+ "Please speak into the microphone")
print("Please speak into the microphone")
record_to_file('emotion.wav')
print("done")
prediction, confidence = deeprec.predict('emotion.wav')
print(f"Prediction: {prediction}")
reply_emotion(prediction)
print("netrual confidence: " + str(confidence[0][0][1]*100) + "%")
print("sad confidence: " + str(confidence[0][0][0]*100) + "%")
print("happy confidence: " + str(confidence[0][0][2]*100) + "%")


# ML algorithm

In [ ]:
from emotion_recognition import EmotionRecognizer
from sklearn.svm import SVC
# init a model, let's use SVC
my_model = SVC()
# pass my model to EmotionRecognizer instance
# and balance the dataset
#     pitch_cepstrum = kwargs.get('pitch_cepstrum')
#     formants = kwargs.get('formants')
#     rmse = kwargs.get('rmse')
#     poly_features = kwargs.get('poly_features')
#     deriv = kwargs.get('deriv')
rec = EmotionRecognizer(model=my_model, emotions=['happy', 'sad'],
                        features=['mfcc', 'chroma','mel','contrast','tonnetz'],
                        balance=True, verbose=1)
# train the model
rec.train()
# check the test accuracy for that model
print("Test score:", rec.test_score())
# check the train accuracy for that model
print("Train score:", rec.train_score())

In [ ]:
# loads the best estimators from `grid` folder that was searched by GridSearchCV in `grid_search.py`,
# and set the model to the best in terms of test score, and then train it
rec.determine_best_model(train=True)
# get the determined sklearn model name
print(rec.model.__class__.__name__, "is the best")
# get the test accuracy score for the best estimator
print("Test score:", rec.test_score())

In [ ]:
os.system("say "+ "Please speak into the microphone")
print("Please speak into the microphone")
record_to_file('emotion.wav')
print("done")
prediction = rec.predict('emotion.wav')
print(f"Prediction: {prediction}")
reply_emotion(prediction)

In [ ]:
if __name__ == "__main__":
    deeprec = DeepEmotionRecognizer(emotions=['sad', 'happy'],
                                    n_rnn_layers=2, n_dense_layers=3, rnn_units=256, dense_units=128,
                                    dropout=0.35,epochs=500)
    # train the model
    deeprec.train(override=False)
    # get the accuracy
    print(deeprec.test_score())

    print("Test accuracy score: {:.3f}%".format(deeprec.test_score()*100))
#     print("Please talk")
    
    
#     filename = "test.wav"
#     record_to_file(filename)
#     result = deeprec.predict(filename)
#     print(result)